In [1]:
!pip install tokenizers
!pip install torchtext
!pip install pytorch_lightning
!pip install datasets
!pip install tensorboard

In [2]:
from config_file import get_config
from train import train_model
import torch
import torch.nn as nn
from torch import optim
from lion_pytorch import Lion


torch.cuda.amp.autocast(enabled=True)

/opt/conda/envs/pytorch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

config = get_config()
config["batch_size"] = 32
config["preload"] = False
config["num_epochs"] = 18


In [4]:
from train import get_model, get_ds, run_validation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device : {device}")

Using device : cuda


In [5]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path
import torchmetrics

from torch.utils.tensorboard import SummaryWriter

Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)


In [6]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
writer = SummaryWriter(config["experiment_name"])
loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr = config["lr"], eps = 1e-9)

# optimizer = Lion(model.parameters(), lr = 1e-4/10, weight_decay=1e-2)


Max length of the source sentence : 309
Max length of the source target : 274
train_dataloader length: 910
val_dataloader length: 3234


In [7]:
MAX_LR = 10**-3
STEPS_PER_EPOCH = len(train_dataloader)
EPOCHS = config["num_epochs"]

In [8]:
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                            max_lr = MAX_LR,
                                            steps_per_epoch=STEPS_PER_EPOCH,
                                            epochs=EPOCHS,
                                            div_factor=100,
                                            three_phase=False,
                                            final_div_factor=100,
                                            anneal_strategy="linear")

In [9]:
train_model(config, EPOCHS,model,train_dataloader,val_dataloader,
             optimizer,loss_fn,tokenizer_src, tokenizer_tgt,scheduler, writer)

Processing Epoch 00: 100%|██████████| 910/910 [02:03<00:00,  7.36it/s, loss=6.194, lr=[0.00019316914308976186]]
stty: standard input: Inappropriate ioctl for device


val_dataloader length :3234
validation batch :9, count: 1
SOURCE He did so.
TARGET Lui ha letto.
PREDICTED Non è la mia .
validation batch :9, count: 2
SOURCE "I was going to say, impassioned: but perhaps you would have misunderstood the word, and been displeased.
TARGET — Stavo per dire appassionata, ma avreste forse interpretato male quella parola e vi sarebbe dispiaciuto.
PREDICTED — Non è un ’ è un ’ , e non è un ’ , e non è un ’ .
validation batch :9, count: 3
SOURCE His brother borrowed money for him, the Princess advised him to return to the country after the wedding, and Oblonsky suggested going abroad.
TARGET Il fratello aveva preso del denaro in prestito per lui, la principessa consigliava di partire per Mosca dopo il matrimonio, Stepan Arkad’ic di andare all’estero.
PREDICTED La sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua sua ’ .

/opt/conda/envs/pytorch21/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/envs/pytorch21/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/envs/pytorch21/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(
Processing Epoch 01: 100%|██████████| 910/910 [02:04<00:00,  7.31it/s, loss=5.660, lr=[0.0003765397923875433]] 
stty: standard input: Inappropriate ioctl for device


val_dataloader length :3234
validation batch :9, count: 1
SOURCE He stopped and for a moment could not understand it.
TARGET Egli si fermò, e a lungo non poté capire.
PREDICTED Egli si mise a lei e non si mise a lei .
validation batch :9, count: 2
SOURCE "You are human and fallible." "I am: so are you--what then?"
TARGET — Siete uomo e fallibile. — Sì, come voi; ebbene?
PREDICTED — Non vi siete stata ? — esclamò , — Jane ?
validation batch :9, count: 3
SOURCE She did not like the bantering tone between Anna and Veslovsky that continued during the game, nor the unnaturalness of grown-up people when they play childish games in the absence of children.
TARGET Non le piacevano i rapporti che intercorrevano fra Vasen’ka Veslovskij e Anna e quella generale innaturalezza dei grandi quando, da soli, senza bambini, fanno un giuoco infantile.
PREDICTED Non si sentiva che , e , e , e , e , e , come , come , come , come si , e si mise a lui , si mise a lui , si mise a lui .
validation batch :9, co

Processing Epoch 02: 100%|██████████| 910/910 [02:04<00:00,  7.31it/s, loss=5.415, lr=[0.0005599104416853247]] 
stty: standard input: Inappropriate ioctl for device


...

val_dataloader length :3234
validation batch :9, count: 1
SOURCE "Fetch that stool," said Mr. Brocklehurst, pointing to a very high one from which a monitor had just risen: it was brought. "Place the child upon it."
TARGET — Avanzate quella seggiola, — disse il signor Bockelhurst accennando una sedia alta, — e metteteci la bambina.
PREDICTED — Com ' è pronta , — disse il signor Bockelhurst , indicando con una porta che aveva chiamato ; — fu contenta di questa bambina , e poi fu contenta di questo .
validation batch :9, count: 2
SOURCE 'Young or old?' asked Levin with a laugh, as Tanya's gestures reminded him of some one.
TARGET — Ma vecchio o giovane? — domandò ridendo Levin, al quale il gesticolare di Tanja aveva ricordato qualcuno.
PREDICTED — Il giovane o del vecchio ? — chiese Levin , sorridendo a un riso di volte che gli venivano incontro .
validation batch :9, count: 3
SOURCE His complexion is too dark for yellows.
TARGET La sua carnagione è troppo scura per i gialli.
PREDICTED È

Processing Epoch 16: 100%|██████████| 910/910 [02:04<00:00,  7.32it/s, loss=1.828, lr=[7.945714285714301e-05]] 
stty: standard input: Inappropriate ioctl for device


val_dataloader length :3234
validation batch :9, count: 1
SOURCE He even seemed offended at our remarks.
TARGET Parve anche offeso dalle nostre osservazioni.
PREDICTED Gli sembrava perfino indifferente alla nostra sciocchezza .
validation batch :9, count: 2
SOURCE "Keep some what?"
TARGET — Dare che?
PREDICTED — Qualche altra cosa ?
validation batch :9, count: 3
SOURCE St. John smiled.
TARGET Saint-John sorrise.
PREDICTED Saint - John sorrise .
validation batch :9, count: 4
SOURCE 'This is Pilate's Admonition – Matthew, chapter xxvii.,' he went on, conscious that his lips were beginning to tremble with excitement; and he stepped behind the visitors.
TARGET Capitolo XXVII del vangelo di Matteo — disse, sentendo che le labbra cominciavano a tremargli per l’agitazione. Si allontanò e si mise dietro di loro.
PREDICTED — Questa è Pilato , l ’ . Matvej , che si , agitando le labbra , capì che erano usciti con l ’ agitazione e che camminava dietro a ospiti .
validation batch :9, count: 5
SOUR

Processing Epoch 17: 100%|██████████| 910/910 [02:04<00:00,  7.33it/s, loss=1.841, lr=[1.0000000000005664e-07]]
stty: standard input: Inappropriate ioctl for device


val_dataloader length :3234
validation batch :9, count: 1
SOURCE During her illness he had learnt to know her thoroughly, had seen into her very soul; and it seemed to him that he had never loved her before.
TARGET L’aveva vista tutta durante la sua malattia; aveva imparato a conoscerne l’anima, e gli pareva di non averla mai amata fino ad allora.
PREDICTED Aveva la malattia di lei che aveva saputo bene , almeno era stato innamorato ed era parso che a fare non volergli bene .
validation batch :9, count: 2
SOURCE 'That would do very well, – or wherever you like,' remarked Ryabinin with contemptuous dignity, as if to show them that though others might find it difficult to know how to behave with different people, yet for him no difficulty of any kind could ever exist.
TARGET — Ma dove a voi piace, signore — disse con aria dignitosa e altera Rjabinin per far intendere che per gli altri potevano esserci difficoltà, sul come e con chi trattare l’affare, ma per lui mai e per nessuna cosa.
PR